In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
# print(tf.__version__)
# print(sys.version_info)
# for module in mpl,np,pd,sklearn,tf,keras:
#     print(module.__name__, module.__version__)

In [ ]:
fashion_minist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_minist.load_data()
x_vaild, x_train = x_train_all[:5000], x_train_all[5000:]
y_vaild, y_train = y_train_all[:5000], y_train_all[5000:]
print(x_vaild.shape, y_vaild.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_vaild = scaler.fit_transform(x_vaild.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test = scaler.fit_transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [ ]:
# 使用tf.keras.sequential构建模型
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
for _ in range(9):
    model.add(keras.layers.Dense(100,activation='relu'))
    model.add(keras.layers.BatchNormalization())
    """
    将匹归一化放在激活函数之前的实现
    model.add(keras.layers.Dense(100))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu))
    """
    
model.add(keras.layers.Dense(100,activation='tanh'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',
             metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Tensorboard , earlystopping 
logdir = './bn-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)

output_model_file = os.path.join(logdir,"fashion_minist_model.h5")
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta = 1e-3)
]
history = model.fit(x_train,y_train,epochs=50,validation_data=(x_vaild,y_vaild),callbacks = callbacks)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize = (8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)